# [ 데이터 수집 과제 제출 ]
- 스타벅스 사이트 -> 지점 정보 추출
- 데이터의 최종 형태 
    - [{'지점명':'GS타워','주소':'서울특별시 강남구 논현로 508 (역삼동)','시군구':'강남구','시도':'서울특별시','타입':'R|G|DT'},...]
- 이런 내용이 나오는 "데이터수집_본인이름.ipynb"  노트 파일 제출
- 3월 10일 24시까지 제출

## 워크플로우 설정
- html 구성을 확인하고 원하는 데이터를 어떻게 크롤링 할 것 인지 확인
- 원하는 데이터를 크롤링 할 방법을 확인 했으면 스텝별로 구분 후 코드 간소화 작업 시작
- 원하는 결과값?
    - dataframe : 리스트안의 딕셔너리 들 '[{ }, { }...]'
    - 데이터를 추출했을때 각 지점별 데이터가 있어야함
- level 4 웹크롤링으로 진행해야 될 것 같음
    - 지역검색을 해야하나 퀵검색을 해야하나???
    - 시도, 시군구 전부 ajax 스타일 확인 - 화면깜빡임 없음, 주소창X
    - 지역검색과 퀵검색이 따로 있음 검색종류를 바꿀때 깜빡임 발생
    - 일단 데이터를 추출해봐야 될것같음.
    - 원하는것은 지점별 정보
    - 이것저것 옵션을 바꿔봤으나 화면 깜빡임이 없었음. 주소창
- 페이지 html을 f12로 확인했을때
    - quickResultLstCon 클래스 안에 정보가 하나씩 들어가있음
    - 하위항목의 result_details 클래스에서 주소와 전화 번호 확인
    
- html 구성확인
    - class"loca_step1" 으로 시/도 를 구분
        - 시/도는 "sido_arae_box" 클래스 하위의 "set_sido_cd_btn" 클래스로 구분 되어있음
    - 시/도를 클릭했을때 clsss"loca_step2"로 구/군을 구분
        - 구/군은 "loca_step2_cont" 클래스 하위의 "set_gugun_cd_btn" 클래스로 구분 되어있음

### 최초 오류에 따른 변경사항
- 퀵검색? 지역검색? 퀵검색 url로 들어가면 시/도 선택 html 크롤링이 안됨
    - "~quick"에서 "https://www.starbucks.co.kr/store/store_map.do?disp=locale" 로 변경
- TODO를 활용하려 했으나 반응형은 하나씩 실행되어 먹히지 않음

In [1]:
# 사전준비 및 가동확인
# 1. 모듈 가져오기
import selenium
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
import numpy as np
import time

In [2]:
# 2. 브라우저 가동 및 접속(지역검색)
driver = wd.Chrome()
TARGET_SITE = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get( TARGET_SITE )
time.sleep(2) 

In [3]:
# 3. 요소 찾기(특정하기)
sido_opts = driver.find_elements( By.CSS_SELECTOR, '.set_sido_cd_btn' )

len(sido_opts)

17

In [4]:
# 제대로 값이 나오는지 확인
# for sido in sido_opts:
#     sido_opt = sido.text
#     print(sido_opt)

In [5]:
# 시/도 리스트
sido_values = [sido_opt.get_attribute('innerText') for sido_opt in sido_opts]

sido_values

['서울',
 '경기',
 '광주',
 '대구',
 '대전',
 '부산',
 '울산',
 '인천',
 '강원',
 '경남',
 '경북',
 '전남',
 '전북',
 '충남',
 '충북',
 '제주',
 '세종']

# 시도후보값 완성
    - 다음은 군/구가 필요
    - 군/구 접근을 위해 시/도를 하나씩 선택해야함
    - 클릭할때마다 값이 바뀌는것을 확인.
    - 시/도를 클릭하고 해당하는 시/도의 구/군만 확인됨(선택 후 0번 인덱스 전체는 필요없으니 슬라이싱[1:])
        - 클래스는 "set_gugun_cd_btn"로 모두 동일, 그러나 다른 시/도를 클릭 할 때 마다 값이 변경됨
        - ajax로 확인 -> 화면갱신됨
        - send_keys 활용해 하나씩 접근이 필요

## 문제발생
    - 왜 send_keys가 안먹히나 봤더니 클릭할때마다 html 부분의 style='dispaly: block" 이부분이
        계속 바뀌는 것을 확인했다. block => none으로 바뀌는걸 보니까 데이터는 열려있으나 보여주느냐 안보여 주느냐 이차이라고 생각됨.
    - 클릭은 되는지확인 => 문제없음 => 클릭으로 진행
    
    - 그냥 사이트 스타일 자체가 클릭으로 접근하게 되어있는듯하다. send_keys로 보내봤자 변하는 부분이 없는것을 확인함. 클릭으로 해결
    - 그러면 어떻게 하나씩 클릭을 해야하나?
    - xpath로 한부분씩 클릭할건지 기존 find_element로 해결할건지 선택해야함
    - xpath
        - li[]의 인덱스값이 다르고 이걸로 구분하는것을 확인(서울은 li[1] 이런식)
    - find_element
        - "set_sido_cd_btn"클래스의 data-sidocd='01' 처럼 data-sidocd 숫자로 구분됨

    ** 클릭외엔 해결방법이 떠오르지 않아 구글링한 결과
        - get_attribute('textContent')의 textContent를 이용해서 HTML 태그를 제외한 텍스트 전부 긁어오기
        - 일단 배운대로 진행
        

In [6]:
# 크롤링 시작
# 시/도 선택이 잘되는지 확인 - 먼저 서울만
# 어떻게 하나씩 접근할것인가? - xpath? css? classname?
# 1. send_keys로 입력해서 들어갈수가없음.
# 2. 클릭을 해야만 각 스텝 클래스의 display 값이 block로 바뀌면서 접근하는 구조로 확인됨
# 3. 따라서 클릭으로 접근을 해야함. => 어떻게 하나씩 접근을 해야하나?
# 4. find_elements로 시/군 리스트를 만들고 for을 이용해 element 하나씩 접근한다.

sido_opts = driver.find_elements( By.CSS_SELECTOR, '.set_sido_cd_btn' )
for i in sido_opts[:1]:
    i.click()
    time.sleep(2)

# 잘 되는것을 확인. time.sleep을 안넣었더니 바로 다음으로 넘어가서 2초 대기시간 설정.

In [7]:
# 시군구 CSS '.set_gugun_cd_btn' 확인 -
sigungu_opts_opts = driver.find_elements( By.CSS_SELECTOR, '.set_gugun_cd_btn' )

len(sigungu_opts_opts)

26

In [8]:
# # 잘불러와지는지 확인
# for sigungu in sigungu_opts[1:]:
#     sigungu_opt = sigungu.text
#     print(sigungu_opt)

In [9]:
sigungu_opts = [sigungu.get_attribute('innerText') for sigungu in sigungu_opts_opts[1:]]
sigungu_opts

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [10]:
for i in sigungu_opts_opts[1:2]:
    i.click()
    time.sleep(1)

In [11]:
# # 지점명 확인
# store_opts_opts = driver.find_elements( By.CSS_SELECTOR, '.quickResultLstCon' )

# store_opts = [store.get_attribute('data-name') for store in store_opts_opts[:]]

# store_opts


In [12]:
# 각 주소 확인
store_opts_address = driver.find_elements( By.CSS_SELECTOR, '.result_details' )
# 슬라이싱을 통해 주소에서 추출한 결과값 중 스타벅스 전화번호는 날림[:-10]
store_address = [store_a.get_attribute('innerText')[:-10] for store_a in store_opts_address[:]]

store_address

['서울특별시 강남구 헌릉로 727 (세곡동)',
 '서울특별시 강남구 역삼로 310 (역삼동)',
 '서울특별시 강남구 논현로175길 38 (신사동)',
 '서울특별시 강남구 삼성로 709 (청담동)',
 '서울특별시 강남구 영동대로96길 12 (삼성동)',
 '서울특별시 강남구 강남대로 512 (논현동)',
 '서울특별시 강남구 테헤란로 311 (역삼동)',
 '서울특별시 강남구 테헤란로 620 (대치동)',
 '서울특별시 강남구 강남대로 396 (역삼동)',
 '서울특별시 강남구 봉은사로 222 (역삼동)',
 '서울특별시 강남구 영동대로 237 (대치동)',
 '서울특별시 강남구 헌릉로569길 18 (세곡동)',
 '서울특별시 강남구 언주로 727 (논현동)',
 '서울특별시 강남구 테헤란로 231 (역삼동)',
 '서울특별시 강남구 영동대로 517 (삼성동)',
 '서울특별시 강남구 광평로 280 (수서동)',
 '서울특별시 강남구 선릉로 34 (개포동)',
 '서울특별시 강남구 선릉로 324 (대치동)',
 '서울특별시 강남구 도산대로 134, 2층 (논현동)',
 '서울특별시 강남구 논현로 648 (논현동)',
 '서울특별시 강남구 선릉로93길 22, 대흥빌딩 1층 (역삼동)',
 '서울특별시 강남구 삼성로86길 7, 1층 (대치동)',
 '서울특별시 강남구 테헤란로 306 (역삼동)',
 '서울특별시 강남구 선릉로 836, 삼원빌딩 1,2층 (청담동)',
 '서울특별시 강남구 테헤란로2길 27, 1층 101호 (역삼동)',
 '서울특별시 강남구 봉은사로 304, 금강빌딩 1-2층 (역삼동)',
 '서울특별시 강남구 언주로174길 29 (신사동)',
 '서울특별시 강남구 도곡로 183 (역삼동)',
 '서울특별시 강남구 영동대로 616 (삼성동)',
 '서울특별시 강남구 테헤란로 334, LG화재 빌딩 (역삼동)',
 '서울특별시 강남구 논현로 841 (신사동)',
 '서울특별시 강남구 논현로 508 (역삼동)',
 '서울특별시 강남구 삼성로 60

In [13]:
# 지점명 확인
store_opts_name = driver.find_elements( By.CSS_SELECTOR, '.quickResultLstCon' )
store_name = [store_a.get_attribute('data-name') for store_a in store_opts_name[:]]

store_name

['강남세곡DT',
 '역삼이마트',
 '압구정현대아파트',
 '청담역',
 '삼성동',
 '강남논현',
 '테헤란로아남타워',
 '삼성역',
 '강남역신분당역사',
 '언주역',
 '대치삼성',
 '강남세곡',
 '도산사거리',
 '센터필드R',
 '코엑스아셈타워',
 '수서역사거리',
 '개포동',
 '대치한티',
 '도산가로수길',
 '논현힐탑',
 '선릉로',
 '포스코',
 '르네상스사거리',
 '압구정로데오역',
 '강남비젼타워',
 '아크로힐스논현',
 '신압구정',
 '역삼럭키',
 '코엑스사거리',
 '선릉역',
 '압구정미소',
 'GS타워',
 '삼성현대힐',
 '선정릉역',
 '학여울역',
 '아크플레이스',
 '포이',
 '대치역',
 '강남구청역',
 '큰길타워',
 '구역삼사거리',
 '압구정로',
 '압구정역',
 '선릉세화빌딩',
 '삼성',
 '신세계푸드마켓도곡',
 '대치',
 '학동역',
 '삼성교',
 '삼성도심공항',
 '청담',
 '강남대로',
 '역삼포스코',
 '역삼대로',
 '가로수길',
 '코엑스몰',
 '역삼초교사거리',
 '한티역',
 '청담사거리',
 '서울세관사거리',
 '도곡역',
 '신사역',
 '을지병원사거리',
 '포스코사거리',
 '대치사거리',
 '학동사거리',
 '강남우성',
 '청담스타R',
 '신사가로수',
 '압구정',
 '청담영동대로',
 '대치은마사거리',
 '강남R',
 '도곡공원',
 '강남구청정문',
 '스타필드코엑스몰R',
 '강남오거리',
 '봉은사로선정릉',
 '선릉동신빌딩R',
 '양재강남빌딩R',
 '수서역R',
 '압구정R',
 '삼성역섬유센터R',
 '코엑스별마당',
 '압구정윤성빌딩',
 '봉은사역',
 '대치재경빌딩',
 '국기원사거리',
 '신사역성일빌딩',
 '논현역사거리',
 '역삼아레나빌딩']

In [14]:
# 타입 확인
store_opts_type = driver.find_elements( By.CSS_SELECTOR, '.quickResultLstCon > i' )
store_type = [store_a.get_attribute('class') for store_a in store_opts_type[:]]
# 클래스 이름으로 타입을 구분하고 있기 때문에 각각 변환해줄 필요

store_types = []
for i in store_type:
    if i == 'pin_generalDT':
        store_types.append('DT')
    elif i == 'pin_reserve':
        store_types.append('R')
    elif i == 'pin_general':
        store_types.append('G')
    elif i == 'pin_generalWT':
        store_types.append('WT')
    else:
        store_types.append('N')

# 다른 타입있는지 고유값 확인
print(np.unique(store_types))
store_types

['DT' 'G' 'R']


['DT',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'R',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'R',
 'G',
 'G',
 'G',
 'G',
 'R',
 'G',
 'G',
 'R',
 'G',
 'G',
 'R',
 'R',
 'R',
 'R',
 'R',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G']

In [15]:
# 함수화
def convert_store_types(store_opts_type):
    store_type = [store_a.get_attribute('class') for store_a in store_opts_type]
    store_types = []

    for i in store_type:
        if i == 'pin_generalDT':
            store_types.append('DT')
        elif i == 'pin_reserve':
            store_types.append('R')
        elif i == 'pin_general':
            store_types.append('G')
        else:
            store_types.append('N')

    return store_types

store_opts_type = driver.find_elements(By.CSS_SELECTOR, '.quickResultLstCon > i')
store_types = convert_store_types(store_opts_type)
store_types

['DT',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'R',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'R',
 'G',
 'G',
 'G',
 'G',
 'R',
 'G',
 'G',
 'R',
 'G',
 'G',
 'R',
 'R',
 'R',
 'R',
 'R',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G']

In [16]:
# 리스트 수 가 일치하는지 확인
if len(store_address) == len(store_name) == len(store_types):
    print('일치')
else:
    print('오류발생')

일치


# 크롤링 성공! 
    - 이제 리스트안의 딕셔너리 형태로 만들기!
    - 함수를 만들어 시도하려했으나 더복잡해져서 취소
    - 리스트 3개와 문자열 2개를 리스트안의 딕셔너리 형태로 바꿔야하니 반복문 활용

In [17]:
# # 함수로 시도 => 오히려 복잡해져서 실패
# store_container = []
# def add_store_info(name, address, type):
#     store_info_new = {}
#     store_info_new["지점명"] = name
#     store_info_new["주소"] = address
#     # store_info_new["시군구"] = sigungu
#     # store_info_new["시도"] = sido
#     store_info_new["타입"] = type
#     store_container.append(store_info_new)

# 원하는 형태를 위한 스텝구성
## 지역검색 => 시/도 => 군/구 => 지점별 데이터 접근 => 군/구로 돌아와서 전체 크롤링 => 시/도도 마찬가지로 돌아와서 전체 크롤링
### step 1
    - 지역검색에서 클릭으로 시/도 접근
### step 2
    - 접근한 시/도에서 클릭으로 군/구 접근
### step 3
    - 지점 데이터를 셀레니움으로 크롤링, 원하는 데이터 저장
### step 4
    - 각 시군구별로 반복해야하기 때문에 이전으로 돌아가기
### step 5
    - 모든 지점 데이터를 뽑기 위해 조건문으로 반복

In [18]:
# 클릭했을때 값 확인
# 시/도 = ".sidoSelectName"
# 군/구 = ".gugunSelectName" * 군/구 앞에 시/도가 붙기 때문에 문자열 슬라이싱
store_sido = driver.find_element( By.CSS_SELECTOR, '.sidoSelectName' ).get_attribute('innerText')
store_sigungu = driver.find_element( By.CSS_SELECTOR, '.gugunSelectName' ).get_attribute('innerText')[3:]
store_sido, store_sigungu

('서울', '강남구')

In [19]:
# 리소스 준비 및 확인
store_types, store_name, store_address, store_sido, store_sigungu

(['DT',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'R',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'R',
  'G',
  'G',
  'G',
  'G',
  'R',
  'G',
  'G',
  'R',
  'G',
  'G',
  'R',
  'R',
  'R',
  'R',
  'R',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G',
  'G'],
 ['강남세곡DT',
  '역삼이마트',
  '압구정현대아파트',
  '청담역',
  '삼성동',
  '강남논현',
  '테헤란로아남타워',
  '삼성역',
  '강남역신분당역사',
  '언주역',
  '대치삼성',
  '강남세곡',
  '도산사거리',
  '센터필드R',
  '코엑스아셈타워',
  '수서역사거리',
  '개포동',
  '대치한티',
  '도산가로수길',
  '논현힐탑',
  '선릉로',
  '포스코',
  '르네상스사거리',
  '압구정로데오역',
  '강남비젼타워',
  '아크로힐스논현',
  '신압구정',
  '역삼럭키',
  '코엑스사거리',
  '선릉역',
  '압구정미소',
  'GS타워',
  '삼성현대힐',


In [20]:
# 리스트 수 가 일치하는지 확인
if len(store_address) == len(store_name) == len(store_types):
    print('일치')
else:
    print('에러 발생')

일치


# 과제 제출 결과값
    - 추후 step별 보충요소를 위해 함수화 진행해 결과값 도출

In [21]:
# 스탭반복을 위해 함수화
def store_containers(store_name, store_address, store_sigungu, store_sido, store_types):
    store_container = []
    for i in range(len(store_address)):
        store_container.append(
            {"지점명"   : store_name[i],
            "주소"     : store_address[i],
            "시군구"   : store_sigungu,
            "시도"     : store_sido,
            "타입"     : store_types[i]
        }
        )
    return store_container

result = store_containers(store_name, store_address, store_sigungu, store_sido, store_types)
print(result)

[{'지점명': '강남세곡DT', '주소': '서울특별시 강남구 헌릉로 727 (세곡동)', '시군구': '강남구', '시도': '서울', '타입': 'DT'}, {'지점명': '역삼이마트', '주소': '서울특별시 강남구 역삼로 310 (역삼동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '압구정현대아파트', '주소': '서울특별시 강남구 논현로175길 38 (신사동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '청담역', '주소': '서울특별시 강남구 삼성로 709 (청담동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '삼성동', '주소': '서울특별시 강남구 영동대로96길 12 (삼성동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '강남논현', '주소': '서울특별시 강남구 강남대로 512 (논현동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '테헤란로아남타워', '주소': '서울특별시 강남구 테헤란로 311 (역삼동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '삼성역', '주소': '서울특별시 강남구 테헤란로 620 (대치동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '강남역신분당역사', '주소': '서울특별시 강남구 강남대로 396 (역삼동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '언주역', '주소': '서울특별시 강남구 봉은사로 222 (역삼동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '대치삼성', '주소': '서울특별시 강남구 영동대로 237 (대치동)', '시군구': '강남구', '시도': '서울', '타입': 'G'}, {'지점명': '강남세곡', '주

# 보충 => 코드 간소화 작업 및 모든 데이터 크롤링

In [22]:
sido_click = driver.find_elements(By.CLASS_NAME, 'set_sido_cd_btn')
back_loc = driver.find_element( By.CSS_SELECTOR, '.on' )
sigungu_click = driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')
back_loc.click()

# 전체 결과값을 result에 저장
result = []

# 지역검색을 클릭해 다시 돌아가기, [2:5]로 테스트값 슬라이싱해 정상적으로 나오는지 결과값 확인
# step 1
for i in sido_click[2:5]:
    i.click()
    time.sleep(1)
    sigungu_click = driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')

    for j in range(1, len(sigungu_click)):
        # step 2
        sigungu_click = driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')
        e = sigungu_click[j]
        e.click()
        time.sleep(2)

        # stale이 계속 변경되기 때문에 변수를 for 루프안에서 다시 지정
        store_opts_name = driver.find_elements( By.CSS_SELECTOR, '.quickResultLstCon' )
        store_name = [store_a.get_attribute('data-name') for store_a in store_opts_name[:]]

        store_opts_address = driver.find_elements( By.CSS_SELECTOR, '.result_details' )
        store_address = [store_a.get_attribute('innerText')[:-10] for store_a in store_opts_address[:]]

        store_sigungu = driver.find_element( By.CSS_SELECTOR, '.gugunSelectName' ).get_attribute('innerText')[3:] 
        store_sido = driver.find_element( By.CSS_SELECTOR, '.sidoSelectName' ).get_attribute('innerText')

        store_opts_type = driver.find_elements(By.CSS_SELECTOR, '.quickResultLstCon > i')
        store_types = convert_store_types(store_opts_type)

        # 리스트 수 가 일치하는지 확인
        if not len(store_address) == len(store_name) == len(store_types):
            print('에러 발생')
        
        # step 3
        result_container = store_containers(store_name, store_address, store_sigungu, store_sido, store_types)
        result = result + result_container
        print(result)
        time.sleep(1)
        
        # step 4
        back_loc.click()
        time.sleep(1)

        i.click()
        time.sleep(1)
        
    # step 5
    back_loc.click()
    time.sleep(1)
        

[{'지점명': '광주송정로', '주소': '광주광역시 광산구 상무대로 202 (송정동)', '시군구': '광산구 ', '시도': '광주', '타입': 'G'}, {'지점명': '광주비아DT', '주소': '광주광역시 광산구 첨단과기로 81 (비아동)', '시군구': '광산구 ', '시도': '광주', '타입': 'DT'}, {'지점명': '광주첨단DT', '주소': '광주광역시 광산구 첨단월봉로 84 (산월동)', '시군구': '광산구 ', '시도': '광주', '타입': 'DT'}, {'지점명': '광주신창강변DT', '주소': '광주광역시 광산구 북문대로 321 (신창동)', '시군구': '광산구 ', '시도': '광주', '타입': 'DT'}, {'지점명': '광산수완', '주소': '광주광역시 광산구 장신로 134, 1~2층 (수완동)', '시군구': '광산구 ', '시도': '광주', '타입': 'G'}, {'지점명': '광주첨단', '주소': '광주광역시 광산구 첨단중앙로 104 (월계동)', '시군구': '광산구 ', '시도': '광주', '타입': 'G'}, {'지점명': '광주하남', '주소': '광주광역시 광산구 사암로216번길 38-16, 1~2층 (우산동)', '시군구': '광산구 ', '시도': '광주', '타입': 'G'}, {'지점명': '광주장덕', '주소': '광주광역시 광산구 장신로 85 (장덕동)', '시군구': '광산구 ', '시도': '광주', '타입': 'G'}, {'지점명': '광주쌍암', '주소': '광주광역시 광산구 첨단중앙로 158 (쌍암동)', '시군구': '광산구 ', '시도': '광주', '타입': 'G'}, {'지점명': '광주신가DT', '주소': '광주광역시 광산구 목련로 335 (신가동)', '시군구': '광산구 ', '시도': '광주', '타입': 'DT'}, {'지점명': '광주수완DT', '주소': '광주광역시 광산구 임방울대로 487 (수완동)', '시군구': '광산구 ', '시도': '광주'

In [23]:
# 테스트 결과 : 광주(전체 63개) + 대구(전체 89개) + 대전(전체 65개) 지점 수 일치!
len(result), result
# 모든 결과값 이상없음!

(217,
 [{'지점명': '광주송정로',
   '주소': '광주광역시 광산구 상무대로 202 (송정동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'G'},
  {'지점명': '광주비아DT',
   '주소': '광주광역시 광산구 첨단과기로 81 (비아동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'DT'},
  {'지점명': '광주첨단DT',
   '주소': '광주광역시 광산구 첨단월봉로 84 (산월동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'DT'},
  {'지점명': '광주신창강변DT',
   '주소': '광주광역시 광산구 북문대로 321 (신창동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'DT'},
  {'지점명': '광산수완',
   '주소': '광주광역시 광산구 장신로 134, 1~2층 (수완동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'G'},
  {'지점명': '광주첨단',
   '주소': '광주광역시 광산구 첨단중앙로 104 (월계동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'G'},
  {'지점명': '광주하남',
   '주소': '광주광역시 광산구 사암로216번길 38-16, 1~2층 (우산동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'G'},
  {'지점명': '광주장덕',
   '주소': '광주광역시 광산구 장신로 85 (장덕동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'G'},
  {'지점명': '광주쌍암',
   '주소': '광주광역시 광산구 첨단중앙로 158 (쌍암동)',
   '시군구': '광산구 ',
   '시도': '광주',
   '타입': 'G'},
  {'지점명': '광주신가DT',
   '주소': '광주광역시 광산

# 오류없이 모든 지점정보 데이터 추출 완료!
    - 현재 모든 지점 데이터를 추출하는 것은 너무 오랜시간이 걸리기 때문에 3개 지역을 슬라이싱하여 테스트
    - for i in sido_click[2:5]: => for i in sido_click[:]: 로 변경해 모든 지점 추출 가능!